In [1]:
!pip install piexif

In [2]:
import os
import csv
import logging
import argparse
from pathlib import Path
import cv2 # OpenCV per lettura/scrittura immagini/video e calcolo nitidezza
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tqdm import tqdm
from PIL import Image, UnidentifiedImageError # Pillow per gestione EXIF
import piexif # Per manipolare EXIF
import shutil # Per copiare/muovere file (utile per pulizia e salvataggio frame
from types import SimpleNamespace

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# try:
#     from googleapiclient.discovery import build
#     from googleapiclient.errors import HttpError
#     from google.oauth2 import service_account
#     GOOGLE_LIBS_AVAILABLE = True
# except ImportError:
#     GOOGLE_LIBS_AVAILABLE = False
#     print("Librerie Google non trovate. Integrazione Sheets disabilitata.")

# --- Configurazione Percorso Base ---
# Prova a montare Drive se in Colab, altrimenti usa percorso locale

In [5]:
#CONFIGURAZIONE PARAMETRI GLOBALI
IMG_SIZE = (224, 224) # Dimensioni target delle immagini
BATCH_SIZE = 32 # Dimensione del batch
EPOCHS_HEAD = 8 # Epoche training testa
EPOCHS_FINE = 4 # Epoche fine-tuning
FRAME_INTERVAL = 1 # Secondi tra frame estratti
LEARN_HEAD_LR = 1e-3 # Learning rate testa
LEARN_FULL_LR = 1e-4 # Learning rate fine-tuning

# Parametri selezione frame video
SHARPNESS_THRESHOLD = 80.0 # Soglia nitidezza
FRAMES_PER_PLACE_PER_VIDEO = 5 # Max frame da salvare

In [6]:
GDRIVE_PROJECT_PATH = Path("/content/drive/MyDrive/ProgettoClassificazioneLuoghi")

# PERCORSI FILE
DEFAULT_TRAIN_DIR = GDRIVE_PROJECT_PATH / "dataset" / "train"  # Giusto
DEFAULT_VAL_DIR = GDRIVE_PROJECT_PATH / "dataset" / "val"    # Giusto

# puntano dentro la cartella 'dataset'
# Assicurarsi che i nomi "inputIMG" e "inputVID" siano corretti
# rispetto a come si chiamano DENTRO la cartella 'dataset'
DEFAULT_IMG_DIR = GDRIVE_PROJECT_PATH / "dataset" / "inputIMG"
DEFAULT_VID_DIR = GDRIVE_PROJECT_PATH / "dataset" / "inputVID"

DEFAULT_OUTPUT = GDRIVE_PROJECT_PATH / "output"
DEFAULT_CSV = DEFAULT_OUTPUT / "results.csv"
PROCESSED_LOG = DEFAULT_OUTPUT / "processed.log"
BEST_FRAMES_OUTPUT_DIR = DEFAULT_OUTPUT / "best_frames"
TEMP_FRAME_EXTRACT_DIR = DEFAULT_OUTPUT / "temp_frames"

# Crea directory di output necessarie
DEFAULT_OUTPUT.mkdir(parents=True, exist_ok=True)
BEST_FRAMES_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
TEMP_FRAME_EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Cartella Immagini Input (Modificata): {DEFAULT_IMG_DIR}")
print(f"Cartella Video Input (Modificata): {DEFAULT_VID_DIR}")
print(f"Cartella Output Generale: {DEFAULT_OUTPUT}")
print(f"Cartella Output Frame Migliori: {BEST_FRAMES_OUTPUT_DIR}")
print(f"Cartella Output Temporanea Frame: {TEMP_FRAME_EXTRACT_DIR}")

Cartella Immagini Input (Modificata): /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG
Cartella Video Input (Modificata): /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputVID
Cartella Output Generale: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output
Cartella Output Frame Migliori: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/best_frames
Cartella Output Temporanea Frame: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/temp_frames


In [7]:
from pathlib import Path

gdrive_base = Path("/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset")
img_input_path_test = gdrive_base / "inputIMG" # Usa il nome ESATTO della tua cartella
vid_input_path_test = gdrive_base / "inputVID" # Usa il nome ESATTO della tua cartella

print(f"Verifica percorso immagini: {img_input_path_test}")
print(f"Esiste? {img_input_path_test.exists()}")
print(f"È una directory? {img_input_path_test.is_dir()}")
if img_input_path_test.exists() and img_input_path_test.is_dir():
    print(f"Contenuto: {list(img_input_path_test.iterdir())[:5]}") # Stampa i primi 5 file/cartelle

print(f"\nVerifica percorso video: {vid_input_path_test}")
print(f"Esiste? {vid_input_path_test.exists()}")
print(f"È una directory? {vid_input_path_test.is_dir()}")
if vid_input_path_test.exists() and vid_input_path_test.is_dir():
    print(f"Contenuto: {list(vid_input_path_test.iterdir())[:5]}")

Verifica percorso immagini: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG
Esiste? True
È una directory? True
Contenuto: [PosixPath('/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di download (2).jpg'), PosixPath('/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di download (3).jpg'), PosixPath('/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di download (1).jpg'), PosixPath('/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di 40_Santa_Maria_delle_Grazie.jpg'), PosixPath('/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di 1630611238845.jpg')]

Verifica percorso video: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputVID
Esiste? True
È una directory? True
Contenuto: [PosixPath('/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputVID/Copia di Copia di timelapse_chiostro.mov'), PosixPath('/content/d

In [8]:
# FUNZIONI PER GOOGLE SHEETS (Opzionale)
  #creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    #if os.path.exists(os.path.abspath('token/token.json')):
        #creds = Credentials.from_authorized_user_file(os.path.abspath('token/token.json'), SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    #if not creds or not creds.valid:
        #if creds and creds.expired and creds.refresh_token:
            #creds.refresh(Request())
        #else:
            #flow = InstalledAppFlow.from_client_secrets_file(
                #os.path.abspath('token/credentials.json'), SCOPES)
            #creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        #with open(os.path.abspath('token/token.json'), 'w') as token:
            #token.write(creds.to_json())
   # return creds


In [9]:
#FUNZIONI PER GESTIONE LOG E FILE DI CONTROLLO
def setup_logging(output_dir: Path):
    """Configura il logging sia su file che su console."""
    log_file = output_dir / "run.log"
    # Rimuove handler esistenti per evitare log duplicati
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s %(levelname)s: %(message)s",
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    logging.info(f"Logging configurato. Log salvati in: {log_file}")

def is_already_processed(key: str) -> bool:
    """Controlla se una chiave è nel file processed.log."""
    try:
        if not PROCESSED_LOG.exists():
            return False
        with open(PROCESSED_LOG, 'r') as f:
           return any(key == line.strip() for line in f)
    except Exception as e:
        logging.error(f"Errore leggendo {PROCESSED_LOG}: {e}")
        return False

def mark_processed(key: str):
    """Aggiunge una chiave al file processed.log."""
    try:
        with open(PROCESSED_LOG, "a") as f:
            f.write(key + "\n")
    except Exception as e:
        logging.error(f"Errore scrivendo su {PROCESSED_LOG}: {e}")

In [10]:
# FUNZIONI PER ESTRAZIONE FRAME E MODIFICA EXIF
def extract_frames(video_path: Path, frame_output_base_dir: Path, interval_sec: int) -> list:
    """Estrae frame da un video, salvandoli in una sottocartella specifica."""
    saved_frames_paths = []
    # Crea sottocartella specifica per i frame di questo video dentro la base dir
    video_frame_dir = frame_output_base_dir / video_path.stem
    video_frame_dir.mkdir(parents=True, exist_ok=True)

    try:
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            logging.error(f"Impossibile aprire il video: {video_path}")
            return []

        fps = cap.get(cv2.CAP_PROP_FPS) or 30
        if fps <= 0: fps = 30
        step = max(1, int(fps * interval_sec)) # Assicura che step sia almeno 1

        frame_count = 0
        saved_frame_index = 0
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        pbar_desc = f"Extracting frames from {video_path.name}"
        with tqdm(total=total_frames, desc=pbar_desc) as pbar:
            while True:
                ret, frame = cap.read()
                if not ret: break

                if frame_count % step == 0:
                    out_filename = f"{video_path.stem}_frame{saved_frame_index:05d}.jpg" # Padding aumentato
                    out_file_path = video_frame_dir / out_filename
                    success_write = cv2.imwrite(str(out_file_path), frame, [cv2.IMWRITE_JPEG_QUALITY, 90])
                    if success_write:
                        saved_frames_paths.append(out_file_path)
                        saved_frame_index += 1
                    else:
                        logging.warning(f"Errore nel salvataggio del frame: {out_file_path}")

                frame_count += 1
                pbar.update(1)

        cap.release()
        logging.info(f"Estratti {len(saved_frames_paths)} frame da {video_path.name} in {video_frame_dir}")

    except Exception as e:
        logging.error(f"Errore durante estrazione frame da {video_path}: {e}")
        if 'cap' in locals() and cap.isOpened(): cap.release()
        # Pulizia parziale in caso di errore?
        # shutil.rmtree(video_frame_dir, ignore_errors=True) # Rimuove cartella anche se errore
        return []

    return saved_frames_paths

def update_image_exif(img_path: Path, label: str):
    """Scrive etichetta nel campo EXIF ImageDescription."""
    if not img_path.is_file():
        logging.warning(f"Impossibile aggiornare EXIF, file non trovato: {img_path}")
        return
    try:
        img = Image.open(img_path)
        try:
           exif_dict = piexif.load(img.info.get('exif', b""))
        except (piexif.InvalidImageDataError, ValueError) as exif_load_err:
            logging.warning(f"Dati EXIF non validi/assenti/corrotti in {img_path.name} ({exif_load_err}). Creazione nuovo dizionario.")
            exif_dict = {'0th': {}, 'Exif': {}, 'GPS': {}, '1st': {}, 'thumbnail': None}

        if '0th' not in exif_dict: exif_dict['0th'] = {}

        exif_dict['0th'][piexif.ImageIFD.ImageDescription] = label.encode('utf-8')

        # Rimuove il thumbnail esistente per sicurezza
        if 'thumbnail' in exif_dict: exif_dict['thumbnail'] = None

        try:
            exif_bytes = piexif.dump(exif_dict)
        except Exception as dump_error:
            logging.error(f"Errore durante piexif.dump per {img_path.name}: {dump_error}")
            img.close()
            return

        try:
           img_format = img.format # Preserva formato originale
           img.save(img_path, exif=exif_bytes, format=img_format, quality=95) # Aggiunta quality
           logging.debug(f"EXIF aggiornato per: {img_path.name} con etichetta '{label}'")
        except Exception as save_error:
            logging.error(f"Errore durante salvataggio {img_path.name} con EXIF: {save_error}")

        img.close()

    except FileNotFoundError:
         logging.warning(f"File non trovato durante tentativo aggiornamento EXIF: {img_path}")
    except UnidentifiedImageError:
         logging.warning(f"PIL non riconosce il formato immagine: {img_path}")
    except Exception as e:
        logging.error(f"Errore generico aggiornamento EXIF per {img_path}: {e}")
        if 'img' in locals() and hasattr(img, 'close'): img.close()

In [11]:
#FUNZIONE PER CALCOLO NITIDEZZA
def calculate_sharpness(image_cv2):
    """Calcola la varianza del Laplaciano per un'immagine OpenCV."""
    if image_cv2 is None or image_cv2.size == 0: return 0.0
    try:
        if image_cv2.ndim == 3: gray = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2GRAY)
        elif image_cv2.ndim == 2: gray = image_cv2
        else: return 0.0
        laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
        return laplacian_var
    except Exception as e:
        logging.error(f"Errore in calculate_sharpness: {e}")
        return 0.0

In [12]:
# Definizione e Preparazione Modello (TensorFlow/Keras)

def build_model(num_classes: int) -> models.Model:
    """Costruisce un modello ResNet50 per fine-tuning."""
    backbone = ResNet50(weights="imagenet", include_top=False, input_shape=(*IMG_SIZE, 3))
    x = layers.GlobalAveragePooling2D(name="gap")(backbone.output)
    x = layers.Dropout(0.5, name="dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="predictions")(x)
    model = models.Model(backbone.input, outputs, name="resnet50_finetuned")

    logging.info(f"Congelamento {len(backbone.layers)} layer del backbone ResNet50.")
    for layer in backbone.layers: layer.trainable = False

    model.compile(optimizer=optimizers.Adam(learning_rate=LEARN_HEAD_LR),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    logging.info("Modello compilato per training testa.")
    return model

# Modifica la funzione make_dataset in questo modo:

def make_dataset(dirpath: Path, shuffle: bool, subset: str = None, validation_split: float = None):
    """Crea un tf.data.Dataset da una directory, con opzione split,
       e restituisce il dataset e i nomi delle classi."""
    if not dirpath.is_dir():
        logging.error(f"Directory dataset non trovata: {dirpath}")
        return None, None # Restituisce None per ds e class_names
    try:
        # Crea il dataset INIZIALE per ottenere class_names
        initial_ds = tf.keras.preprocessing.image_dataset_from_directory(
            dirpath,
            labels="inferred",
            label_mode="categorical", # Importante per num_classes
            batch_size=BATCH_SIZE,    # Il batch size qui non influenza class_names
            image_size=IMG_SIZE,      # Nemmeno image_size
            shuffle=shuffle,          # Shuffle qui non è cruciale per class_names
            seed=123,
            validation_split=validation_split,
            subset=subset
        )
        if not initial_ds:
           logging.error(f"image_dataset_from_directory (initial) ha restituito None per {dirpath}")
           return None, None

        class_names = initial_ds.class_names # Ottieni i nomi delle classi QUI
        logging.info(f"Dataset creato da {dirpath}. Classi trovate: {class_names}")

        # Ora puoi applicare map e prefetch come prima sull'initial_ds
        # o ricreare il dataset se preferisci, ma usare initial_ds è efficiente
        processed_ds = initial_ds.map(lambda imgs, labs: (preprocess_input(imgs), labs),
                                       num_parallel_calls=tf.data.AUTOTUNE)
        processed_ds = processed_ds.prefetch(tf.data.AUTOTUNE)

        return processed_ds, class_names # Restituisce entrambi

    except Exception as e:
        logging.error(f"Errore creazione dataset da {dirpath}: {e}")
        return None, None

In [13]:
def train(args):
    """Esegue il ciclo di training completo."""
    setup_logging(args.output) # Assicurati che setup_logging sia definita
    logging.info("--- Inizio Processo di Training ---")

    # Gestione dataset training/validazione
    validation_split_needed = 0.2
    train_dir_path = Path(args.train_dir)
    val_dir_path = Path(args.val_dir) if args.val_dir else None # Può essere stringa vuota o None

    class_names_list = None # Inizializza per i nomi delle classi

    if val_dir_path and val_dir_path.is_dir(): # Se val_dir è un percorso valido e una directory
        logging.info(f"Uso directory separata per validazione: {val_dir_path}")
        train_ds, class_names_list = make_dataset(train_dir_path, shuffle=True)
        # Per val_ds, i class_names dovrebbero essere gli stessi, quindi non li riassegniamo
        val_ds, _ = make_dataset(val_dir_path, shuffle=False)
    else:
        if val_dir_path: # Se era specificato ma non valido
             logging.warning(f"Directory di validazione '{val_dir_path}' non trovata o non valida.")
        logging.info(f"Uso {validation_split_needed*100}% split da {train_dir_path} per validazione.")
        train_ds, class_names_list = make_dataset(train_dir_path, shuffle=True, subset='training', validation_split=validation_split_needed)
        val_ds, _ = make_dataset(train_dir_path, shuffle=False, subset='validation', validation_split=validation_split_needed)

    # Controlla se i dataset e i nomi delle classi sono stati creati correttamente
    if train_ds is None or val_ds is None or class_names_list is None:
         logging.error("Creazione dataset o recupero nomi classi fallita. Training interrotto.")
         return

    num_classes = len(class_names_list)
    logging.info(f"Numero classi: {num_classes} - Nomi: {class_names_list}")
    if num_classes < 2:
        logging.error("Richieste almeno 2 classi per il training. Training interrotto.")
        return

    # Costruzione modello
    # Assicurati che build_model sia definita e usi le costanti globali corrette
    model = build_model(num_classes)

    # Callback
    checkpoint_head_path = args.output / "model_head_best.h5"
    checkpoint_fine_path = args.output / "model_fine_tuned_best.h5"

    # Salva solo i pesi se vuoi più flessibilità nel ricostruire il modello,
    # altrimenti save_weights_only=False salva l'intero modello (architettura + pesi + stato optimizer)
    model_checkpoint_head = tf.keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_head_path), # Keras preferisce stringhe per i percorsi
        save_weights_only=False,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
    )
    model_checkpoint_fine = tf.keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_fine_path),
        save_weights_only=False,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
    )
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5, # Numero di epoche senza miglioramenti prima di fermarsi
        restore_best_weights=True # Ripristina i pesi dell'epoca con la miglior val_loss
    )

    # Training testa
    logging.info(f"--- Training testa per {args.epochs_head} epoche (LR={args.learn_head_lr}) ---")
    # Assicurati che args.epochs_head e args.learn_head_lr siano accessibili e corretti
    model.fit(train_ds, validation_data=val_ds, epochs=args.epochs_head, callbacks=[model_checkpoint_head])
    logging.info("Training testa completato.")

    # Carica pesi migliori testa (se il modello è stato salvato interamente, questo ricarica il miglior modello)
    if checkpoint_head_path.exists():
        logging.info(f"Caricamento del miglior modello dal training della testa da {checkpoint_head_path}")
        model = models.load_model(checkpoint_head_path) # Carica l'intero modello, non solo i pesi
    else:
        logging.warning("Checkpoint del training testa non trovato. Si procede con gli ultimi pesi del modello attuale.")

    # Sblocco layer per fine-tuning
    logging.info("--- Fine-tuning completo: Sblocco layer ---")
    # Esempio: sblocca tutti i layer del backbone.
    # si potrebbe voler sbloccare solo una parte per fine-tuning più conservativo.
    # In genere, i layer di BatchNormalization si tengono congelati quando si fa fine-tuning
    # se il backbone è stato pre-allenato con essi.
    for layer in model.layers: # Itera su tutti i layer, incluso il backbone
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True
        # Se si vogliono sbloccare anche i layer di Batch Normalization del backbone:
        # else:
        # layer.trainable = True # Attenzione: può destabilizzare se il batch size è piccolo

    # In alternativa, un modo comune è sbloccare i layer del backbone originale
    # if hasattr(model, 'get_layer') and model.get_layer(name='resnet50'): # Assumendo che il backbone si chiami 'resnet50'
    #    backbone = model.get_layer(name='resnet50') # Ottieni il backbone
    #    # Sblocca gli ultimi N blocchi del backbone, ad esempio
    #    fine_tune_at = 100 # Numero di layer da sbloccare dalla fine
    #    for layer in backbone.layers[-fine_tune_at:]:
    #        if not isinstance(layer, layers.BatchNormalization):
    #            layer.trainable = True

    logging.info(f"Layer trainabili dopo sblocco: {sum(1 for l in model.layers if l.trainable)}")

    # Ricompilare con LR basso
    # Assicurarsi che args.learn_full_lr sia accessibile e corretto
    model.compile(optimizer=optimizers.Adam(learning_rate=args.learn_full_lr),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    logging.info(f"Modello ricompilato per fine-tuning (LR={args.learn_full_lr})")

    # Fine-tuning
    # Assicurarsi che args.epochs_fine sia accessibile e corretto
    logging.info(f"--- Fine-tuning per max {args.epochs_fine} epoche (LR={args.learn_full_lr}) ---")
    model.fit(train_ds, validation_data=val_ds, epochs=args.epochs_fine, callbacks=[model_checkpoint_fine, early_stopping])
    logging.info("Fine-tuning completato.")

    # Salvataggio modello finale
    final_model_path = args.output / "place_model.h5"
    # Se early_stopping ha ripristinato i pesi migliori e model_checkpoint_fine ha salvato
    # il modello in quell'epoca, allora checkpoint_fine_path è il migliore.
    # Altrimenti, se early_stopping non è scattato ma save_best_only sì,
    # checkpoint_fine_path è comunque il migliore di quel run.
    # Se nessun checkpoint è stato salvato (es. solo 1 epoca e non abbastanza buona),
    # allora salviamo lo stato attuale del modello.
    if checkpoint_fine_path.exists():
        logging.info(f"Salvataggio del miglior modello dal fine-tuning ({checkpoint_fine_path}) come modello finale.")
        # Se checkpoint_fine_path è stato salvato, dovrebbe essere il modello migliore o l'ultimo.
        # Rinominiamo per chiarezza se è il migliore, altrimenti si salva l'ultimo stato del modello.
        # Poiché early_stopping ha restore_best_weights=True, model CONTIENE GIÀ i pesi migliori
        # al termine di model.fit se early stopping è scattato.
        # Quindi, possiamo semplicemente salvare 'model'.
        # ModelCheckpoint con save_best_only=True potrebbe aver già salvato il migliore in checkpoint_fine_path.
        # Se early_stopping ha ripristinato i pesi e model_checkpoint_fine non ha salvato *quella precisa* epoca,
        # allora model è meglio di checkpoint_fine_path.
        # La logica più sicura è: se EarlyStopping ha funzionato, 'model' ha i pesi migliori.
        # Se ModelCheckpoint ha salvato qualcosa, potrebbe essere l'ultima migliore epoca.
        # Priorità a EarlyStopping se ha ripristinato.
        if early_stopping.stopped_epoch > 0 and early_stopping.restore_best_weights:
            logging.info("Early stopping ha ripristinato i pesi migliori. Salvataggio dello stato corrente del modello.")
            model.save(final_model_path)
        elif checkpoint_fine_path.exists():
            logging.info(f"Uso del checkpoint salvato da ModelCheckpoint: {checkpoint_fine_path}")
            checkpoint_fine_path.rename(final_model_path)
        else:
            logging.warning("Nessun checkpoint di fine-tuning salvato e EarlyStopping non ha ripristinato pesi. Salvataggio dello stato attuale del modello.")
            model.save(final_model_path)
    else: # Se il file di checkpoint non esiste affatto
        logging.info("Nessun checkpoint di fine-tuning trovato. Salvataggio dello stato attuale del modello.")
        model.save(final_model_path)

    logging.info(f"Modello finale salvato in: {final_model_path}")

    # Salvataggio mappatura classi usando class_names_list
    class_indices_path = args.output / "class_indices.csv"
    try:
        pd.DataFrame({"class_name": class_names_list, "index": list(range(num_classes))}).to_csv(class_indices_path, index=False)
        logging.info(f"Mappatura classi salvata in: {class_indices_path}")
    except Exception as e:
        logging.error(f"Errore durante il salvataggio della mappatura classi: {e}")

    logging.info("--- Processo di Training Concluso ---")

In [14]:
# Funzione di Inferenza (Corretta)

def infer(args):
    """Esegue inferenza su immagini e video."""
    setup_logging(args.output)
    logging.info("--- Inizio Processo di Inferenza ---")

    # Caricamento modello e classi
    model_path = args.output / "place_model.h5"
    class_indices_path = args.output / "class_indices.csv"
    if not model_path.exists() or not class_indices_path.exists():
        logging.error(f"Modello ({model_path}) o indici classi ({class_indices_path}) non trovati.")
        return

    try:
        model = models.load_model(model_path)
        logging.info(f"Modello caricato da {model_path}")
        idx_df = pd.read_csv(class_indices_path)
        class_names = idx_df.sort_values("index")["class_name"].tolist()
        logging.info(f"Classi caricate: {class_names}")
    except Exception as e:
        logging.error(f"Errore caricamento modello o classi: {e}")
        return

         #Preparazione file CSV output
    csv_fieldnames = ["SourceType", "PredictedPlace", "OriginalPath", "Confidence", "IsVideo", "BestFrameSavedPath", "Sharpness"]
    try:
        csv_file = open(args.csv_path, "a", newline="", encoding='utf-8')
        writer = csv.DictWriter(csv_file, fieldnames=csv_fieldnames)
        if csv_file.tell() == 0: # Scrive header se file è vuoto
            writer.writeheader()
        logging.info(f"File CSV pronto: {args.csv_path}")
    except Exception as e:
         logging.error(f"Impossibile aprire/scrivere file CSV {args.csv_path}: {e}")
         return

    # Funzione Helper per caricare/preprocessare immagine per inferenza
    # TENERE SOLO QUESTA DEFINIZIONE
    def load_and_preprocess_img_infer(path_str: str):
        # logging.debug(f"Tentativo caricamento frame: {path_str}")
        try:
            img_bytes = tf.io.read_file(path_str)
            # logging.debug(f"  --> Letto {len(img_bytes)} bytes da {path_str}")
            img = tf.image.decode_jpeg(img_bytes, channels=3)
            # logging.debug(f"  --> Decodificato frame {path_str}, shape: {img.shape}")
            img = tf.image.resize(img, IMG_SIZE)
            # logging.debug(f"  --> Ridimensionato frame {path_str}")
            img = preprocess_input(img)
            # logging.debug(f"  --> Preprocessato frame {path_str}")
            return img
        except tf.errors.InvalidArgumentError as e:
             logging.error(f"TF ArgumentError per {path_str}: {e}")
             return None
        except Exception as e:
            logging.error(f"Errore generico caricamento/prep immagine {path_str}: {e}")
            return None

    # INFERENZA IMMAGINI SINGOLE
    logging.info(f"Inferenza immagini in: {args.img_dir}")
    img_dir = Path(args.img_dir)
    if not img_dir.is_dir():
         logging.warning(f"Directory immagini input non trovata: {img_dir}")
    else:
        img_files = sorted(list(img_dir.glob("*.jpg")) + list(img_dir.glob("*.png")) + list(img_dir.glob("*.jpeg")))
        logging.info(f"Trovate {len(img_files)} immagini.")

        valid_img_paths_str = []
        original_img_paths = []

        print("\nDEBUG: Inizio ciclo filtro immagini...")
        for i, p in enumerate(img_files): # NUOVO CICLO CON DEBUG
            key = f"IMG::{p.resolve()}"
            print(f"DEBUG {i+1}/{len(img_files)}: Processando '{p.name}' - Key: '{key}'")

            already_processed = is_already_processed(key)
            print(f"DEBUG {i+1}: Risultato is_already_processed: {already_processed}")
            if already_processed:
                print(f"DEBUG {i+1}: Immagine saltata perché già processata.")
                continue

            print(f"DEBUG {i+1}: Tentativo tf.io.read_file...")
            try:
                tf_read_success = False
                _ = tf.io.read_file(str(p))
                tf_read_success = True
                print(f"DEBUG {i+1}: tf.io.read_file OK.")
                valid_img_paths_str.append(str(p))
                original_img_paths.append(p)
                print(f"DEBUG {i+1}: Aggiunta alle liste valide.")
            except Exception as e:
                print(f"DEBUG {i+1}: Eccezione durante tf.io.read_file: {e}")
                logging.warning(f"Immagine corrotta/non leggibile (saltata): {p.name} ({e})")

        # Controllo finale liste e continuazione
        print(f"\nDEBUG: Fine ciclo filtro immagini.")
        print(f"DEBUG: Lunghezza valid_img_paths_str: {len(valid_img_paths_str)}")
        print(f"DEBUG: Lunghezza original_img_paths: {len(original_img_paths)}")

        if valid_img_paths_str:
            print("DEBUG: Entrando nel blocco 'if valid_img_paths_str:'. Inizio creazione dataset immagini.")
            img_dataset = tf.data.Dataset.from_tensor_slices(valid_img_paths_str)
            img_dataset = img_dataset.map(load_and_preprocess_img_infer, num_parallel_calls=tf.data.AUTOTUNE).filter(lambda x: x is not None)
            img_dataset = img_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

            logging.info(f"Predizioni su {len(original_img_paths)} nuove immagini valide...")
            all_predictions = model.predict(img_dataset)

            # Ciclo per processare le predizioni (con i debug print che hai già inserito)
            print("DEBUG: Inizio ciclo processamento predizioni immagini...") # Aggiunto per chiarezza
            for i, pred in enumerate(all_predictions):
                print(f"\nDEBUG Immagine {i}: {original_img_paths[i].name}")
                print(f"DEBUG: Predizione grezza (pred): {pred}")
                print(f"DEBUG: Tipo di pred: {type(pred)}")
                print(f"DEBUG: Shape di pred: {pred.shape}")

                label_idx = np.argmax(pred)
                raw_confidence = pred[label_idx]
                print(f"DEBUG: Valore confidenza grezza (pred[label_idx]): {raw_confidence}") # Già aggiunto
                confidence = float(raw_confidence)
                label = class_names[label_idx]

                print(f"DEBUG: Indice={label_idx}, Confidenza calcolata={confidence}, Etichetta={label}") # Già aggiunto

                img_path = original_img_paths[i]
                key = f"IMG::{img_path.resolve()}"

                logging.info(f"Img: {img_path.name} -> Classe: {label} (Conf: {confidence:.3f})")

                csv_row = {
                    "SourceType": "Image", "PredictedPlace": label,
                    "OriginalPath": str(img_path.resolve()), "Confidence": f"{confidence:.4f}",
                    "IsVideo": 0, "BestFrameSavedPath": "N/A", "Sharpness": "N/A"
                }
                writer.writerow(csv_row)

                update_image_exif(img_path, label)
                mark_processed(key)
            print("DEBUG: Fine ciclo processamento predizioni immagini.") # Aggiunto per chiarezza

            logging.info(f"Completata inferenza su {len(original_img_paths)} immagini.")
        else:
             print("DEBUG: Entrando nel blocco 'else'. Nessuna nuova immagine valida trovata.")
             logging.info("Nessuna nuova immagine valida trovata.")

              # INFERENZA VIDEO
    logging.info(f"Inferenza video in: {args.vid_dir}")
    vid_dir = Path(args.vid_dir)
    if not vid_dir.is_dir():
         logging.warning(f"Directory video input non trovata: {vid_dir}")
    else:
        video_files = sorted(list(vid_dir.glob("*.mp4")) + list(vid_dir.glob("*.mov")) + list(vid_dir.glob("*.avi")))
        logging.info(f"Trovati {len(video_files)} video.")

        for vid_path in video_files:
            key = f"VID::{vid_path.resolve()}"
            if is_already_processed(key):
                logging.info(f"Video già processato (saltato): {vid_path.name}")
                continue

            logging.info(f"Processando video: {vid_path.name}")
            current_video_temp_frame_dir = TEMP_FRAME_EXTRACT_DIR / vid_path.stem

             # 1. Estrazione Frame
            frame_paths = extract_frames(vid_path, TEMP_FRAME_EXTRACT_DIR, args.frame_interval)
            if not frame_paths:
                logging.warning(f"Errore o nessun frame estratto da {vid_path.name}. Salto.")
                continue

                 # 2. Predizione sui frame
            frame_paths_str = [str(p) for p in frame_paths]
            if not frame_paths_str: # Doppia verifica
                 logging.warning(f"Lista path frame vuota per {vid_path.name}. Salto.")
                 continue

            frame_dataset = tf.data.Dataset.from_tensor_slices(frame_paths_str)
            frame_dataset = frame_dataset.map(load_and_preprocess_img_infer, num_parallel_calls=tf.data.AUTOTUNE).filter(lambda x: x is not None)
            # Ricalcola numero effettivo di frame validi
            valid_frame_count = tf.data.experimental.cardinality(frame_dataset).numpy()
            if valid_frame_count <= 0:
                logging.warning(f"Nessun frame valido dopo preprocessing per {vid_path.name}. Salto.")
                shutil.rmtree(current_video_temp_frame_dir, ignore_errors=True) # Pulisci
                continue

            frame_dataset = frame_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
            logging.info(f"Predizioni su {valid_frame_count} frame validi...")

            try:
                all_predictions = model.predict(frame_dataset)
                if all_predictions.shape[0] != valid_frame_count:
                    logging.warning(f"Numero predizioni ({all_predictions.shape[0]}) non corrisponde ai frame validi ({valid_frame_count}) per {vid_path.name}. Potrebbero esserci stati errori nel map.")
                    # Potresti voler interrompere qui o procedere con cautela
                all_pred_indices = np.argmax(all_predictions, axis=1)
                all_pred_confidences = np.max(all_predictions, axis=1)
            except Exception as pred_err:
                 logging.error(f"Errore predizione frame {vid_path.name}: {pred_err}")
                 shutil.rmtree(current_video_temp_frame_dir, ignore_errors=True) # Pulisci
                 continue
                  # 3. Determina Classe Dominante
            if len(all_pred_indices) == 0:
                 logging.warning(f"Nessuna predizione valida per {vid_path.name}. Salto.")
                 shutil.rmtree(current_video_temp_frame_dir, ignore_errors=True) # Pulisci
                 continue

            majority_idx = np.bincount(all_pred_indices).argmax()
            video_label = class_names[majority_idx]
            majority_confidences = all_pred_confidences[all_pred_indices == majority_idx]
            video_confidence = float(np.mean(majority_confidences)) if len(majority_confidences) > 0 else 0.0
            logging.info(f"Video: {vid_path.name} -> Classe: {video_label} (Conf. media: {video_confidence:.3f})")

              # 4. Seleziona Frame Migliori
            candidate_frames_info = []
            # Associa predizioni ai path originali (potrebbe essere imperfetto se filter() ha rimosso frame)
            # Un modo più robusto sarebbe fare `predict` su ogni frame singolarmente, ma più lento.
            # Tentativo di associazione basato sull'ordine (funziona se filter non rimuove nulla)
            if len(frame_paths) == len(all_pred_indices):
                for i, frame_path in enumerate(frame_paths):
                    if all_pred_indices[i] == majority_idx:
                        try:
                            frame_img_cv2 = cv2.imread(str(frame_path))
                            if frame_img_cv2 is None: continue
                            sharpness = calculate_sharpness(frame_img_cv2)
                            if sharpness >= args.sharpness_threshold:
                                candidate_frames_info.append((sharpness, frame_path, all_pred_confidences[i]))
                        except Exception as e:
                             logging.error(f"Errore analisi nitidezza frame {frame_path}: {e}")
            else:
                 logging.warning(f"Numero predizioni non corrisponde a numero frame estratti per {vid_path.name}. Selezione frame migliori potrebbe essere imprecisa.")
                 # Fallback: analizza tutti i frame estratti individualmente se necessario (più lento)


            candidate_frames_info.sort(key=lambda x: x[0], reverse=True) # Ordina per nitidezza
            best_frames_to_save = candidate_frames_info[:args.frames_per_place]
            logging.info(f"Selezionati {len(best_frames_to_save)} frame candidati (sharp >= {args.sharpness_threshold}).")

            # 5. Salva Frame Migliori e Aggiorna EXIF
            saved_best_frame_paths = []
            if best_frames_to_save:
                video_best_frames_dir = BEST_FRAMES_OUTPUT_DIR / video_label / vid_path.stem
                video_best_frames_dir.mkdir(parents=True, exist_ok=True)

                for sharpness, frame_path, frame_conf in best_frames_to_save:
                    frame_index_str = Path(frame_path).stem.split('frame')[-1] # Estrae numero frame
                    out_filename = f"{vid_path.stem}__{video_label}__frame{frame_index_str}_sharp{sharpness:.0f}_conf{frame_conf:.3f}.jpg"
                    out_path = video_best_frames_dir / out_filename
                    try:
                        shutil.copy(frame_path, out_path) # Copia il frame
                        update_image_exif(out_path, video_label) # Aggiorna EXIF della copia
                        saved_best_frame_paths.append(str(out_path.resolve()))
                    except Exception as e:
                        logging.error(f"Errore salvataggio/EXIF frame migliore {out_path}: {e}")

            # 6. Scrittura CSV
            csv_row = {
                "SourceType": "Video", "PredictedPlace": video_label,
                "OriginalPath": str(vid_path.resolve()), "Confidence": f"{video_confidence:.4f}",
                "IsVideo": 1,
                "BestFrameSavedPath": ";".join(saved_best_frame_paths) if saved_best_frame_paths else "N/A",
                "Sharpness": f"{best_frames_to_save[0][0]:.1f}" if best_frames_to_save else "N/A"
            }
            writer.writerow(csv_row)

              # 7. Aggiorna Sheets (Opzionale)
            # if creds:
            #     sheet_row = ["Video", video_label, str(vid_path.resolve()), pd.Timestamp.now().isoformat()]
            #     append_row_to_sheet(creds, args.spreadsheet_id, args.sheet_name, sheet_row)

            # 8. Marca video come processato
            mark_processed(key)

            # 9. Pulizia frame temporanei
            logging.info(f"Pulizia frame temporanei per {vid_path.name}...")
            shutil.rmtree(current_video_temp_frame_dir, ignore_errors=True)

    #Fine Ciclo Video
    logging.info("Inferenza Video Completata")

    # Chiusura file CSV
    try: csv_file.close()
    except Exception as e: logging.error(f"Errore chiusura CSV: {e}")

    logging.info("Processo di Inferenza Concluso")


In [15]:
# Setup Google Sheets (Opzionale)
    # creds = None
    # service_account_path = GDRIVE_PROJECT_PATH / 'service_account.json'
    # if args.spreadsheet_id and GOOGLE_LIBS_AVAILABLE:
    #     logging.info("Configurazione Google Sheets...")
    #     creds = getCredentials(service_account_path)
    #     if creds:
    #         # ... (eventuale scrittura header e validazione dati) ...
    #         logging.info("Google Sheets configurato.")
    #     else:
    #         logging.warning("Credenziali Sheets non caricate. Integrazione disabilitata.")
    # else:
    #      logging.info("Spreadsheet ID non fornito o librerie Google mancanti. Integrazione Sheets disabilitata.")

In [16]:
# Blocco Principale (Esecuzione Train o Infer)

if __name__ == "__main__":
    #Configurazione Manuale Argomenti
    # Utile per esecuzione diretta in notebook/IDE
    class Args: pass
    args = Args()

    #IMPOSTA LA MODALITÀ: 'train' o 'infer'
    args.mode = "infer"

    # Percorsi (derivati da GDRIVE_PROJECT_PATH)
    args.train_dir = str(DEFAULT_TRAIN_DIR)
    args.val_dir = str(DEFAULT_VAL_DIR) # Se non esiste, train() usa split
    args.img_dir = str(DEFAULT_IMG_DIR)
    args.vid_dir = str(DEFAULT_VID_DIR)
    args.output = DEFAULT_OUTPUT
    args.csv_path = DEFAULT_CSV

    # Parametri training/inferenza
    args.epochs_head = EPOCHS_HEAD
    args.epochs_fine = EPOCHS_FINE
    args.learn_head_lr = LEARN_HEAD_LR
    args.learn_full_lr = LEARN_FULL_LR
    args.frame_interval = FRAME_INTERVAL
    args.sharpness_threshold = SHARPNESS_THRESHOLD
    args.frames_per_place = FRAMES_PER_PLACE_PER_VIDEO

    # Parametri Google Sheets (Opzionale)
    args.spreadsheet_id = None # Es: "YOUR_SPREADSHEET_ID"
    args.sheet_name = "RisultatiInferenza" # Nome foglio

    #Esecuzione
    if args.mode == "train":
        train(args)
    elif args.mode == "infer":
        infer(args)
    else:
        print(f"Errore: Modalità '{args.mode}' non riconosciuta.")

    print("\nEsecuzione Blocco Principale Terminata")

2025-05-09 07:45:51,906 INFO: Logging configurato. Log salvati in: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/run.log
2025-05-09 07:45:52,717 INFO: --- Inizio Processo di Inferenza ---
2025-05-09 07:45:58,847 WARNING: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-05-09 07:46:01,288 INFO: Modello caricato da /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/place_model.h5
2025-05-09 07:46:02,451 INFO: Classi caricate: ['chiostro_esterno', 'ingegneria_esterno']
2025-05-09 07:46:02,455 INFO: File CSV pronto: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/results.csv
2025-05-09 07:46:02,456 INFO: Inferenza immagini in: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG
2025-05-09 07:46:02,461 INFO: Trovate 5 immagini.



DEBUG: Inizio ciclo filtro immagini...
DEBUG 1/5: Processando 'Copia di 1630611238845.jpg' - Key: 'IMG::/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di 1630611238845.jpg'
DEBUG 1: Risultato is_already_processed: False
DEBUG 1: Tentativo tf.io.read_file...
DEBUG 1: tf.io.read_file OK.
DEBUG 1: Aggiunta alle liste valide.
DEBUG 2/5: Processando 'Copia di 40_Santa_Maria_delle_Grazie.jpg' - Key: 'IMG::/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di 40_Santa_Maria_delle_Grazie.jpg'
DEBUG 2: Risultato is_already_processed: False
DEBUG 2: Tentativo tf.io.read_file...
DEBUG 2: tf.io.read_file OK.
DEBUG 2: Aggiunta alle liste valide.
DEBUG 3/5: Processando 'Copia di download (1).jpg' - Key: 'IMG::/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di download (1).jpg'
DEBUG 3: Risultato is_already_processed: False
DEBUG 3: Tentativo tf.io.read_file...
DEBUG 3: tf.io.read_file OK.
DEBUG 3: Aggiunta alle liste 

2025-05-09 07:46:07,994 INFO: Predizioni su 5 nuove immagini valide...


DEBUG 5: tf.io.read_file OK.
DEBUG 5: Aggiunta alle liste valide.

DEBUG: Fine ciclo filtro immagini.
DEBUG: Lunghezza valid_img_paths_str: 5
DEBUG: Lunghezza original_img_paths: 5
DEBUG: Entrando nel blocco 'if valid_img_paths_str:'. Inizio creazione dataset immagini.
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
2025-05-09 07:46:14,529 INFO: Img: Copia di 1630611238845.jpg -> Classe: ingegneria_esterno (Conf: 1.000)
2025-05-09 07:46:14,572 WARNING: File non trovato durante tentativo aggiornamento EXIF: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di 1630611238845.jpg
2025-05-09 07:46:14,580 INFO: Img: Copia di 40_Santa_Maria_delle_Grazie.jpg -> Classe: ingegneria_esterno (Conf: 1.000)
2025-05-09 07:46:14,584 WARNING: File non trovato durante tentativo aggiornamento EXIF: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di 40_Santa_Maria_delle_Grazie.jpg
2025-05-09 07:46:14,590 INFO: Img: Copia d

DEBUG: Inizio ciclo processamento predizioni immagini...

DEBUG Immagine 0: Copia di 1630611238845.jpg
DEBUG: Predizione grezza (pred): [1.4702787e-04 9.9985290e-01]
DEBUG: Tipo di pred: <class 'numpy.ndarray'>
DEBUG: Shape di pred: (2,)
DEBUG: Valore confidenza grezza (pred[label_idx]): 0.9998528957366943
DEBUG: Indice=1, Confidenza calcolata=0.9998528957366943, Etichetta=ingegneria_esterno

DEBUG Immagine 1: Copia di 40_Santa_Maria_delle_Grazie.jpg
DEBUG: Predizione grezza (pred): [3.5119410e-05 9.9996483e-01]
DEBUG: Tipo di pred: <class 'numpy.ndarray'>
DEBUG: Shape di pred: (2,)
DEBUG: Valore confidenza grezza (pred[label_idx]): 0.9999648332595825
DEBUG: Indice=1, Confidenza calcolata=0.9999648332595825, Etichetta=ingegneria_esterno

DEBUG Immagine 2: Copia di download (1).jpg
DEBUG: Predizione grezza (pred): [2.4646604e-06 9.9999750e-01]
DEBUG: Tipo di pred: <class 'numpy.ndarray'>
DEBUG: Shape di pred: (2,)
DEBUG: Valore confidenza grezza (pred[label_idx]): 0.9999974966049194
DEB

Extracting frames from Copia di Copia di timelapse_chiostro.mov: 100%|██████████| 766/766 [00:50<00:00, 15.11it/s]
2025-05-09 07:47:10,053 INFO: Estratti 16 frame da Copia di Copia di timelapse_chiostro.mov in /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/temp_frames/Copia di Copia di timelapse_chiostro
2025-05-09 07:47:10,120 WARNING: Nessun frame valido dopo preprocessing per Copia di Copia di timelapse_chiostro.mov. Salto.
2025-05-09 07:47:10,148 INFO: Processando video: Copia di Copia di timelapse_chiostro_2.mp4
Extracting frames from Copia di Copia di timelapse_chiostro_2.mp4: 100%|██████████| 450/450 [00:36<00:00, 12.49it/s]
2025-05-09 07:47:47,734 INFO: Estratti 18 frame da Copia di Copia di timelapse_chiostro_2.mp4 in /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/temp_frames/Copia di Copia di timelapse_chiostro_2
2025-05-09 07:47:47,762 WARNING: Nessun frame valido dopo preprocessing per Copia di Copia di timelapse_chiostro_2.mp4. Salto.
2025-05-09 0


Esecuzione Blocco Principale Terminata


In [17]:
# BLOCCO DI CONFIGURAZIONE PER LA MODALITÀ 'TRAIN'
# Questo blocco crea un oggetto 'args' che simula gli argomenti
# che verrebbero passati da riga di comando allo script.

# Verifica che le costanti globali e i percorsi di default siano definiti.
# Se hai eseguito le celle precedenti, dovrebbero esserlo.
# Esempio di controllo (opzionale, ma utile per debug):
# try:
#     print(f"Valore di DEFAULT_TRAIN_DIR: {DEFAULT_TRAIN_DIR}")
#     print(f"Valore di DEFAULT_OUTPUT: {DEFAULT_OUTPUT}")
#     print(f"Valore di EPOCHS_HEAD: {EPOCHS_HEAD}")
# except NameError as e:
#     print(f"ERRORE: Una costante o percorso di default non è definito: {e}")
#     print("Assicurati di aver eseguito tutte le celle di configurazione precedenti.")
#     raise # Interrompe l'esecuzione se mancano


# Creazione dell'oggetto args specifico per il training
args_train_config = SimpleNamespace(
    # Percorsi per i dati di training e validazione
    # Usa i percorsi di default che dovrebbero essere già stati configurati
    # per puntare al tuo Google Drive.
    train_dir=str(DEFAULT_TRAIN_DIR),
    val_dir=str(DEFAULT_VAL_DIR),   # La funzione train() gestirà il caso in cui
                                    # questa directory non esista o sia None,
                                    # usando uno split dal train_dir.

    # Percorso per l'output del training (modello, log, ecc.)
    output=DEFAULT_OUTPUT,

    # Parametri per le epoche di training
    epochs_head=EPOCHS_HEAD,
    epochs_fine=EPOCHS_FINE,

    # Learning rates (anche se LEARN_HEAD_LR è usato globalmente in build_model,
    # includerlo qui è buona pratica se si volesse modificare build_model)
    learn_head_lr=LEARN_HEAD_LR,
    learn_full_lr=LEARN_FULL_LR
)

# Stampa di verifica dei parametri impostati per il training (opzionale)
print("--- Parametri per il Training ---")
print(f"Directory Training: {args_train_config.train_dir}")
print(f"Directory Validazione: {args_train_config.val_dir}")
print(f"Directory Output: {args_train_config.output}")
print(f"Epoche Training Testa: {args_train_config.epochs_head}")
print(f"Epoche Fine-Tuning: {args_train_config.epochs_fine}")
print(f"Learning Rate Testa: {args_train_config.learn_head_lr}")
print(f"Learning Rate Fine-Tuning: {args_train_config.learn_full_lr}")
print("---------------------------------")

# Chiamata alla funzione di training
print("\nAvvio del processo di training...")
try:
    # Assicurarsi che la funzione 'train' sia definita
    train(args_train_config)
    print("\nProcesso di training terminato.")
except NameError:
    print("ERRORE: La funzione 'train' non è definita. Assicurati di aver eseguito la cella con la sua definizione.")
except Exception as e:
    print(f"ERRORE INASPETTATO durante il training: {e}")
    # Per un traceback più dettagliato in caso di errore
    # traceback.print_exc()

2025-05-09 07:47:47,803 INFO: Logging configurato. Log salvati in: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/run.log
2025-05-09 07:47:47,839 INFO: --- Inizio Processo di Training ---
2025-05-09 07:47:47,843 INFO: Uso directory separata per validazione: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/val


--- Parametri per il Training ---
Directory Training: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/train
Directory Validazione: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/val
Directory Output: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output
Epoche Training Testa: 8
Epoche Fine-Tuning: 4
Learning Rate Testa: 0.001
Learning Rate Fine-Tuning: 0.0001
---------------------------------

Avvio del processo di training...
Found 65 files belonging to 2 classes.


2025-05-09 07:47:49,798 INFO: Dataset creato da /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/train. Classi trovate: ['chiostro_esterno', 'ingegneria_esterno']


Found 27 files belonging to 2 classes.


2025-05-09 07:47:50,417 INFO: Dataset creato da /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/val. Classi trovate: ['chiostro_esterno', 'ingegneria_esterno']
2025-05-09 07:47:50,433 INFO: Numero classi: 2 - Nomi: ['chiostro_esterno', 'ingegneria_esterno']


...

3/3 ━━━━━━━━━━━━━━━━━━━━ 58s 9s/step - accuracy: 1.0000 - loss: 0.0067 - val_accuracy: 0.4074 - val_loss: 29.7159
Epoch 2/4
2/3 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.9531 - loss: 1.4491

2025-05-09 07:52:54,474 WARNING: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - accuracy: 0.9612 - loss: 1.2002 - val_accuracy: 0.6296 - val_loss: 1.3805
Epoch 3/4
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6002 - loss: 2.1300 - val_accuracy: 0.4074 - val_loss: 12.1533
Epoch 4/4
3/3 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.9690 - loss: 0.3458 - val_accuracy: 0.4074 - val_loss: 11.5240


2025-05-09 07:53:17,692 INFO: Fine-tuning completato.
2025-05-09 07:53:17,720 INFO: Salvataggio del miglior modello dal fine-tuning (/content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/model_fine_tuned_best.h5) come modello finale.
2025-05-09 07:53:17,722 INFO: Uso del checkpoint salvato da ModelCheckpoint: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/model_fine_tuned_best.h5
2025-05-09 07:53:17,729 INFO: Modello finale salvato in: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/place_model.h5
2025-05-09 07:53:17,742 INFO: Mappatura classi salvata in: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/class_indices.csv
2025-05-09 07:53:17,743 INFO: --- Processo di Training Concluso ---



Processo di training terminato.


In [18]:
#Configurazione per la MODALITÀ 'INFER'
args_infer_config = SimpleNamespace(
    # Percorsi per i dati di input per l'inferenza
    # Usa i percorsi di default che dovrebbero essere già stati configurati
    img_dir=str(DEFAULT_IMG_DIR),
    vid_dir=str(DEFAULT_VID_DIR),

    # Percorso dove si trova il modello allenato e dove salvare i risultati
    output=DEFAULT_OUTPUT,
    csv_path=DEFAULT_CSV, # Percorso per il file CSV dei risultati

    # Parametri specifici dell'inferenza
    # Questi dovrebbero usare le costanti globali che hai definito all'inizio
    frame_interval=FRAME_INTERVAL,
    sharpness_threshold=SHARPNESS_THRESHOLD,
    frames_per_place=FRAMES_PER_PLACE_PER_VIDEO,

    # Parametri Google Sheets (se vuoi riattivarli)
    # Lascia a None se non li usi
    spreadsheet_id=None, # Esempio: "IL_TUO_SPREADSHEET_ID_QUI"
    sheet_name="RisultatiInferenza" # Nome del foglio su cui scrivere
)

# Stampa di verifica dei parametri impostati per l'inferenza
print("Parametri per l'Inferenza:")
print(f"Directory Immagini Input: {args_infer_config.img_dir}")
print(f"Directory Video Input: {args_infer_config.vid_dir}")
print(f"Directory Output (modello/risultati): {args_infer_config.output}")
print(f"File CSV Risultati: {args_infer_config.csv_path}")
print(f"Intervallo Frame Video (s): {args_infer_config.frame_interval}")
print(f"Soglia Nitidezza Frame: {args_infer_config.sharpness_threshold}")
print(f"Frame Migliori da Salvare per Luogo/Video: {args_infer_config.frames_per_place}")
if args_infer_config.spreadsheet_id:
    print(f"Spreadsheet ID: {args_infer_config.spreadsheet_id}")
    print(f"Sheet Name: {args_infer_config.sheet_name}")


# Chiamata alla funzione di inferenza
print("\nAvvio del processo di inferenza...")
try:
    # Assicurarsi che la funzione 'infer' sia definita
    infer(args_infer_config)
    print("\nProcesso di inferenza terminato.")
except NameError as e:
    print(f"ERRORE: La funzione 'infer' (o una dipendenza) non è definita: {e}. Assicurati di aver eseguito tutte le celle di definizione.")
except Exception as e:
    print(f"ERRORE INASPETTATO durante l'inferenza: {e}")
    # Per un traceback più dettagliato in caso di errore
import traceback
traceback.print_exc()

2025-05-09 07:53:17,776 INFO: Logging configurato. Log salvati in: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/run.log
2025-05-09 07:53:17,778 INFO: --- Inizio Processo di Inferenza ---


Parametri per l'Inferenza:
Directory Immagini Input: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG
Directory Video Input: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputVID
Directory Output (modello/risultati): /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output
File CSV Risultati: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/results.csv
Intervallo Frame Video (s): 1
Soglia Nitidezza Frame: 80.0
Frame Migliori da Salvare per Luogo/Video: 5

Avvio del processo di inferenza...


2025-05-09 07:53:18,857 WARNING: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-05-09 07:53:18,892 INFO: Modello caricato da /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/place_model.h5
2025-05-09 07:53:18,897 INFO: Classi caricate: ['chiostro_esterno', 'ingegneria_esterno']
2025-05-09 07:53:18,902 INFO: File CSV pronto: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/results.csv
2025-05-09 07:53:18,904 INFO: Inferenza immagini in: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG
2025-05-09 07:53:18,910 INFO: Trovate 5 immagini.
2025-05-09 07:53:18,924 INFO: Nessuna nuova immagine valida trovata.
2025-05-09 07:53:18,925 INFO: Inferenza video in: /content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputVID
2025-05-09 07:53:18,928 INFO: Trovati 2 video.
2025-05-09 07:53:18,931 INFO: Processando video: Copia di Copia di timela


DEBUG: Inizio ciclo filtro immagini...
DEBUG 1/5: Processando 'Copia di 1630611238845.jpg' - Key: 'IMG::/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di 1630611238845.jpg'
DEBUG 1: Risultato is_already_processed: True
DEBUG 1: Immagine saltata perché già processata.
DEBUG 2/5: Processando 'Copia di 40_Santa_Maria_delle_Grazie.jpg' - Key: 'IMG::/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di 40_Santa_Maria_delle_Grazie.jpg'
DEBUG 2: Risultato is_already_processed: True
DEBUG 2: Immagine saltata perché già processata.
DEBUG 3/5: Processando 'Copia di download (1).jpg' - Key: 'IMG::/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di download (1).jpg'
DEBUG 3: Risultato is_already_processed: True
DEBUG 3: Immagine saltata perché già processata.
DEBUG 4/5: Processando 'Copia di download (2).jpg' - Key: 'IMG::/content/drive/MyDrive/ProgettoClassificazioneLuoghi/dataset/inputIMG/Copia di download (2).jpg'

Extracting frames from Copia di Copia di timelapse_chiostro.mov: 100%|██████████| 766/766 [01:03<00:00, 12.13it/s]
2025-05-09 07:54:22,231 INFO: Estratti 16 frame da Copia di Copia di timelapse_chiostro.mov in /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/temp_frames/Copia di Copia di timelapse_chiostro
2025-05-09 07:54:22,263 WARNING: Nessun frame valido dopo preprocessing per Copia di Copia di timelapse_chiostro.mov. Salto.
2025-05-09 07:54:22,290 INFO: Processando video: Copia di Copia di timelapse_chiostro_2.mp4
Extracting frames from Copia di Copia di timelapse_chiostro_2.mp4: 100%|██████████| 450/450 [00:34<00:00, 12.87it/s]
2025-05-09 07:54:57,523 INFO: Estratti 18 frame da Copia di Copia di timelapse_chiostro_2.mp4 in /content/drive/MyDrive/ProgettoClassificazioneLuoghi/output/temp_frames/Copia di Copia di timelapse_chiostro_2
2025-05-09 07:54:57,563 WARNING: Nessun frame valido dopo preprocessing per Copia di Copia di timelapse_chiostro_2.mp4. Salto.
2025-05-09 0


Processo di inferenza terminato.


NoneType: None
